# BBC Text Representations - Classification

**Roll Number:** SE22UARI195

**Tasks:**
1. Load all 9 representations (4 sparse + 5 dense)
2. Train Logistic Regression with C tuning on DEV
3. Evaluate on TEST: Macro-F1 (primary) and Accuracy
4. Generate preds_test.csv for best representation
5. Compare all methods

---

## 1. Setup & Imports

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import pickle
import json
from pathlib import Path
import time

# Scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, classification_report

# Progress bar
from tqdm.notebook import tqdm

# Warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports successful!")

In [ ]:
# Configuration
ROLL = "SE22UARI195"
CACHE_DIR = Path("../cache")
MODELS_DIR = Path("../models")
OUTPUTS_DIR = Path("../outputs")

# Create outputs directory
OUTPUTS_DIR.mkdir(exist_ok=True)

print(f"Roll Number: {ROLL}")
print(f"Cache Directory: {CACHE_DIR}")
print(f"Models Directory: {MODELS_DIR}")
print(f"Outputs Directory: {OUTPUTS_DIR}")

## 2. Load Preprocessed Data & Labels

In [ ]:
print("📂 Loading preprocessed data...\n")

with open(CACHE_DIR / 'train_processed.pkl', 'rb') as f:
    train_df = pickle.load(f)

with open(CACHE_DIR / 'dev_processed.pkl', 'rb') as f:
    dev_df = pickle.load(f)

with open(CACHE_DIR / 'test_processed.pkl', 'rb') as f:
    test_df = pickle.load(f)

# Extract labels
y_train = train_df['label'].values
y_dev = dev_df['label'].values
y_test = test_df['label'].values

print(f"✅ TRAIN: {len(train_df)} documents, {len(set(y_train))} classes")
print(f"✅ DEV: {len(dev_df)} documents")
print(f"✅ TEST: {len(test_df)} documents")
print(f"\nClasses: {sorted(set(y_train))}")

## 3. Load All Representations

In [ ]:
print("\n📊 Loading sparse representations...\n")

# Load sparse matrices
from scipy.sparse import load_npz

representations = {}

# One-Hot Encoding
representations['ohe'] = {
    'train': load_npz(MODELS_DIR / 'X_train_ohe.npz'),
    'dev': load_npz(MODELS_DIR / 'X_dev_ohe.npz'),
    'test': load_npz(MODELS_DIR / 'X_test_ohe.npz')
}
print(f"✅ OHE: {representations['ohe']['train'].shape}")

# Bag-of-Words
representations['bow'] = {
    'train': load_npz(MODELS_DIR / 'X_train_bow.npz'),
    'dev': load_npz(MODELS_DIR / 'X_dev_bow.npz'),
    'test': load_npz(MODELS_DIR / 'X_test_bow.npz')
}
print(f"✅ BOW: {representations['bow']['train'].shape}")

# N-grams
representations['ngram'] = {
    'train': load_npz(MODELS_DIR / 'X_train_ngram.npz'),
    'dev': load_npz(MODELS_DIR / 'X_dev_ngram.npz'),
    'test': load_npz(MODELS_DIR / 'X_test_ngram.npz')
}
print(f"✅ N-grams: {representations['ngram']['train'].shape}")

# TF-IDF
representations['tfidf'] = {
    'train': load_npz(MODELS_DIR / 'X_train_tfidf.npz'),
    'dev': load_npz(MODELS_DIR / 'X_dev_tfidf.npz'),
    'test': load_npz(MODELS_DIR / 'X_test_tfidf.npz')
}
print(f"✅ TF-IDF: {representations['tfidf']['train'].shape}")

In [ ]:
print("\n📊 Loading dense representations...\n")

# Word2Vec Skip-gram NS
representations['w2v_sg_ns'] = {
    'train': np.load(MODELS_DIR / 'X_train_w2v_sg_ns.npy'),
    'dev': np.load(MODELS_DIR / 'X_dev_w2v_sg_ns.npy'),
    'test': np.load(MODELS_DIR / 'X_test_w2v_sg_ns.npy')
}
print(f"✅ W2V Skip-gram NS: {representations['w2v_sg_ns']['train'].shape}")

# Word2Vec CBOW NS
representations['w2v_cbow_ns'] = {
    'train': np.load(MODELS_DIR / 'X_train_w2v_cbow_ns.npy'),
    'dev': np.load(MODELS_DIR / 'X_dev_w2v_cbow_ns.npy'),
    'test': np.load(MODELS_DIR / 'X_test_w2v_cbow_ns.npy')
}
print(f"✅ W2V CBOW NS: {representations['w2v_cbow_ns']['train'].shape}")

# Word2Vec Skip-gram HS
representations['w2v_sg_hs'] = {
    'train': np.load(MODELS_DIR / 'X_train_w2v_sg_hs.npy'),
    'dev': np.load(MODELS_DIR / 'X_dev_w2v_sg_hs.npy'),
    'test': np.load(MODELS_DIR / 'X_test_w2v_sg_hs.npy')
}
print(f"✅ W2V Skip-gram HS: {representations['w2v_sg_hs']['train'].shape}")

# Word2Vec CBOW HS
representations['w2v_cbow_hs'] = {
    'train': np.load(MODELS_DIR / 'X_train_w2v_cbow_hs.npy'),
    'dev': np.load(MODELS_DIR / 'X_dev_w2v_cbow_hs.npy'),
    'test': np.load(MODELS_DIR / 'X_test_w2v_cbow_hs.npy')
}
print(f"✅ W2V CBOW HS: {representations['w2v_cbow_hs']['train'].shape}")

# GloVe
representations['glove'] = {
    'train': np.load(MODELS_DIR / 'X_train_glove.npy'),
    'dev': np.load(MODELS_DIR / 'X_dev_glove.npy'),
    'test': np.load(MODELS_DIR / 'X_test_glove.npy')
}
print(f"✅ GloVe: {representations['glove']['train'].shape}")

## 4. Classification Helper Functions

In [ ]:
def train_and_evaluate(X_train, y_train, X_dev, y_dev, X_test, y_test, 
                       method_name, C_values=[0.01, 0.1, 1.0, 10.0, 100.0]):
    """
    Train Logistic Regression with hyperparameter tuning on DEV.
    Report best performance on TEST.
    
    Returns:
        dict with best_C, macro_f1, accuracy, predictions
    """
    print(f"\n{'='*60}")
    print(f"Training: {method_name}")
    print(f"{'='*60}")
    
    best_C = None
    best_f1_dev = -1
    best_model = None
    
    # Tune C on DEV set
    print(f"\n🔍 Tuning hyperparameter C on DEV set...")
    for C in C_values:
        clf = LogisticRegression(
            C=C,
            max_iter=1000,
            random_state=42,
            solver='lbfgs',
            multi_class='multinomial'
        )
        
        clf.fit(X_train, y_train)
        y_pred_dev = clf.predict(X_dev)
        
        f1_dev = f1_score(y_dev, y_pred_dev, average='macro')
        acc_dev = accuracy_score(y_dev, y_pred_dev)
        
        print(f"  C={C:7.2f} -> DEV Macro-F1: {f1_dev:.4f}, Accuracy: {acc_dev:.4f}")
        
        if f1_dev > best_f1_dev:
            best_f1_dev = f1_dev
            best_C = C
            best_model = clf
    
    print(f"\n✅ Best C: {best_C} (DEV Macro-F1: {best_f1_dev:.4f})")
    
    # Evaluate on TEST set
    print(f"\n📊 Evaluating on TEST set...")
    y_pred_test = best_model.predict(X_test)
    
    f1_test = f1_score(y_test, y_pred_test, average='macro')
    acc_test = accuracy_score(y_test, y_pred_test)
    
    print(f"\n🎯 TEST Results:")
    print(f"   Macro-F1: {f1_test:.4f}")
    print(f"   Accuracy: {acc_test:.4f}")
    
    # Per-class results
    print(f"\n📋 Classification Report:")
    print(classification_report(y_test, y_pred_test, digits=4))
    
    return {
        'method': method_name,
        'best_C': best_C,
        'macro_f1': float(f1_test),
        'accuracy': float(acc_test),
        'predictions': y_pred_test
    }

print("✅ Helper functions defined!")

## 5. Train Classifiers on All Representations

In [ ]:
print("\n" + "="*80)
print("TRAINING CLASSIFIERS ON ALL REPRESENTATIONS")
print("="*80)

results_classification = {}

### 5.1 Sparse Methods

In [ ]:
# One-Hot Encoding
result = train_and_evaluate(
    representations['ohe']['train'], y_train,
    representations['ohe']['dev'], y_dev,
    representations['ohe']['test'], y_test,
    method_name='One-Hot Encoding (OHE)'
)
results_classification['ohe'] = {
    'macro_f1': result['macro_f1'],
    'accuracy': result['accuracy']
}

In [ ]:
# Bag-of-Words
result = train_and_evaluate(
    representations['bow']['train'], y_train,
    representations['bow']['dev'], y_dev,
    representations['bow']['test'], y_test,
    method_name='Bag-of-Words (BOW)'
)
results_classification['bow'] = {
    'macro_f1': result['macro_f1'],
    'accuracy': result['accuracy']
}

In [ ]:
# N-grams
result = train_and_evaluate(
    representations['ngram']['train'], y_train,
    representations['ngram']['dev'], y_dev,
    representations['ngram']['test'], y_test,
    method_name='N-grams (1,2)'
)
results_classification['ngram'] = {
    'macro_f1': result['macro_f1'],
    'accuracy': result['accuracy']
}

In [ ]:
# TF-IDF (save predictions for submission)
result = train_and_evaluate(
    representations['tfidf']['train'], y_train,
    representations['tfidf']['dev'], y_dev,
    representations['tfidf']['test'], y_test,
    method_name='TF-IDF'
)
results_classification['tfidf'] = {
    'macro_f1': result['macro_f1'],
    'accuracy': result['accuracy']
}

# Save TF-IDF predictions
tfidf_predictions = result['predictions']

### 5.2 Dense Methods (TF-IDF Weighted)

In [ ]:
# Word2Vec Skip-gram NS
result = train_and_evaluate(
    representations['w2v_sg_ns']['train'], y_train,
    representations['w2v_sg_ns']['dev'], y_dev,
    representations['w2v_sg_ns']['test'], y_test,
    method_name='Word2Vec Skip-gram NS + TF-IDF'
)
results_classification['w2v_ns_tfidf'] = {
    'macro_f1': result['macro_f1'],
    'accuracy': result['accuracy']
}

In [ ]:
# Word2Vec CBOW NS (optional - for comparison)
result = train_and_evaluate(
    representations['w2v_cbow_ns']['train'], y_train,
    representations['w2v_cbow_ns']['dev'], y_dev,
    representations['w2v_cbow_ns']['test'], y_test,
    method_name='Word2Vec CBOW NS + TF-IDF'
)
# Note: Not required in results.json but good for comparison

In [ ]:
# Word2Vec Skip-gram HS
result = train_and_evaluate(
    representations['w2v_sg_hs']['train'], y_train,
    representations['w2v_sg_hs']['dev'], y_dev,
    representations['w2v_sg_hs']['test'], y_test,
    method_name='Word2Vec Skip-gram HS + TF-IDF'
)
results_classification['w2v_hs_tfidf'] = {
    'macro_f1': result['macro_f1'],
    'accuracy': result['accuracy']
}

In [ ]:
# Word2Vec CBOW HS (optional - for comparison)
result = train_and_evaluate(
    representations['w2v_cbow_hs']['train'], y_train,
    representations['w2v_cbow_hs']['dev'], y_dev,
    representations['w2v_cbow_hs']['test'], y_test,
    method_name='Word2Vec CBOW HS + TF-IDF'
)
# Note: Not required in results.json but good for comparison

In [ ]:
# GloVe
result = train_and_evaluate(
    representations['glove']['train'], y_train,
    representations['glove']['dev'], y_dev,
    representations['glove']['test'], y_test,
    method_name='GloVe + TF-IDF'
)
results_classification['glove_tfidf'] = {
    'macro_f1': result['macro_f1'],
    'accuracy': result['accuracy']
}

## 6. Summary Comparison

In [ ]:
print("\n" + "="*80)
print("CLASSIFICATION RESULTS SUMMARY")
print("="*80)

print(f"\n{'Method':<30} {'Macro-F1':<12} {'Accuracy':<12}")
print("-"*80)

# Sort by Macro-F1
sorted_results = sorted(results_classification.items(), 
                       key=lambda x: x[1]['macro_f1'], 
                       reverse=True)

for method, metrics in sorted_results:
    print(f"{method:<30} {metrics['macro_f1']:<12.4f} {metrics['accuracy']:<12.4f}")

print("\n" + "="*80)

# Find best method
best_method = sorted_results[0][0]
best_f1 = sorted_results[0][1]['macro_f1']
print(f"\n🏆 Best Method: {best_method} (Macro-F1: {best_f1:.4f})")
print("="*80)

## 7. Generate preds_test.csv

In [ ]:
print("\n💾 Generating preds_test.csv...")

# Create predictions DataFrame
preds_df = pd.DataFrame({
    'id': test_df['id'].values,
    'pred': tfidf_predictions
})

# Save to outputs
preds_df.to_csv(OUTPUTS_DIR / 'preds_test.csv', index=False)

print(f"✅ Saved: {OUTPUTS_DIR / 'preds_test.csv'}")
print(f"   Shape: {preds_df.shape}")
print(f"\nFirst 5 predictions:")
print(preds_df.head())

## 8. Save Classification Results

In [ ]:
# Save classification results
with open(CACHE_DIR / 'classification_results.json', 'w') as f:
    json.dump(results_classification, f, indent=2)

print("\n💾 Classification results saved to cache/classification_results.json")

In [ ]:
print("\n🎉 Notebook 04: Classification - COMPLETE!")
print("\nNext steps:")
print("  1. Run notebook 05: Retrieval")
print("  2. Generate deterministic queries")
print("  3. Calculate MAP@5, Recall@10, Negation Top-1%")
print("  4. Merge all notebooks into final submission")